# Hotel Chatbot

This notebook is based on the Hotel Chatbot problem. How can we provide better customer service at the front desk of a hotel? One simple way is to have an FAQ chatbot that is capable of answering simple questions about the hotel experience. There are many advantages to having a chatbot. 

1. It increases the appeal of the hotel and also increases information throughput.
2. It creates a way to gather questions about the hotel in a data table form. 

In this notebook, we will be taking a look at 2 different models. The cosine similarity modela and the doc2vec model.


## 1. Adding knowledge base to your chatbot

The Chatbot's ability to converse is defined by the data available to it. Take a look at the Ques.txt file to find questions and ans.txt file to find answers to those questions. This chatbot will essentially run a cosine similarity on a question posed with the question bank and try to find an answer. 

Let us begin by importing relevant libraries.

In [1]:
import nltk # to process text data
import numpy as np # to represent corpus as arrays
import random 
import operator
import string # to process standard python strings
from sklearn.metrics.pairwise import cosine_similarity # We will use this later to decide how similar two sentences are
from sklearn.feature_extraction.text import TfidfVectorizer # Remember when you built a function to create a tfidf bag of words in Experience 2? This function does the same thing!

In [2]:
import os
#filepath=os.getcwd()+r'\\hotelChat\\[Dataset] Module27 (ans).txt'
filepath=os.getcwd()+r'\\[Dataset] Module27 (ans).txt'
corpus=open(filepath,'r',errors = 'ignore')
raw_data_ans=corpus.read()
print (raw_data_ans)
filepath=os.getcwd()+'\\[Dataset] Module27(ques).txt'
corpus=open(filepath,'r',errors = 'ignore')
raw_data=corpus.read()
print (raw_data)

My name is Andi.

AXL1.

I am 1 year old.

I am 1 year old.

No. Do you know?.

I'm single.

I'm single.

I can't. I am a bot.

Yes. That's why We are here talking.

Not that I am aware of.

I don't know, you tell me.

I don't have one. I am not human.

Oh please, you are worse.

Wrong place, sport.

Si, puedo hablar espaÃ±ol.

Sim, eu falo portugues.

3 linguas.

3 languages.

I sell them to uncle Mark.

That's top secret.

Yes, we do.

Yes.

Cyber world.

The weather here is amazing today. Full of 0 and 1.

Do I look like a LinkedIn bot.

Girls are always expensive.

Baby, I was born smart.

Yes, sure. Write 'bye' to turn me off.

yes.

no.

fine.

ok.

lazy.

boring.

no.

I am always studying.

YOLO.

Yes.

I do not know. I am not human.

What is food.

yes.

yes.

no?

no.
What is you name?

What is your surname?

How old are you?

What is your age?

Do you know a joke?

Are you single?

Are you married?

Will you marry me?

Do you like people?

Are you part of the Matrix?

Are yo

#  Conversion to lower case

We will convert all text to lower case first. Remember to inspect the result once we are done.

In [3]:
raw_data=raw_data.lower()# converts to lowercase
print (raw_data)

what is you name?

what is your surname?

how old are you?

what is your age?

do you know a joke?

are you single?

are you married?

will you marry me?

do you like people?

are you part of the matrix?

are you beautiful?

tell me about your personality?

youâ€™re annoying?

i want to speak to a human?

can you speak spanish?

can you speak portuguese?

quantas linguas voce consegue falar?

how many languages can you speak?

what do you do with my data?

who made you?

where do you live?

whatâ€™s the weather like today?

do you have a job for me?

are you expensive?

do you get smarter?

can we talk later?

are you fine?

are you fine?

how is your day?

how is your day?

how is your day?

how is your day?

are you real?

are you a student?

what do you rhink abou life?

do you like it classes?

do you like it classes?

what is your favorite food?

yes?

yes

no?

no?

ok?

ok?


####  Segmentation, Lematization and tokenization

We will convert all text to lower case first. Remember to inspect the result once we are done.

In [4]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
sent_tokens = nltk.sent_tokenize(raw_data)# converts documents to list of sentences 

print(sent_tokens)

['what is you name?', 'what is your surname?', 'how old are you?', 'what is your age?', 'do you know a joke?', 'are you single?', 'are you married?', 'will you marry me?', 'do you like people?', 'are you part of the matrix?', 'are you beautiful?', 'tell me about your personality?', 'youâ€™re annoying?', 'i want to speak to a human?', 'can you speak spanish?', 'can you speak portuguese?', 'quantas linguas voce consegue falar?', 'how many languages can you speak?', 'what do you do with my data?', 'who made you?', 'where do you live?', 'whatâ€™s the weather like today?', 'do you have a job for me?', 'are you expensive?', 'do you get smarter?', 'can we talk later?', 'are you fine?', 'are you fine?', 'how is your day?', 'how is your day?', 'how is your day?', 'how is your day?', 'are you real?', 'are you a student?', 'what do you rhink abou life?', 'do you like it classes?', 'do you like it classes?', 'what is your favorite food?', 'yes?', 'yes\n\nno?', 'no?', 'ok?', 'ok?']


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\calaz\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\calaz\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
sent_tokens_ans = nltk.sent_tokenize(raw_data_ans)# converts documents to list of sentences 
print(sent_tokens_ans)

['My name is Andi.', 'AXL1.', 'I am 1 year old.', 'I am 1 year old.', 'No.', 'Do you know?.', "I'm single.", "I'm single.", "I can't.", 'I am a bot.', 'Yes.', "That's why We are here talking.", 'Not that I am aware of.', "I don't know, you tell me.", "I don't have one.", 'I am not human.', 'Oh please, you are worse.', 'Wrong place, sport.', 'Si, puedo hablar espaÃ±ol.', 'Sim, eu falo portugues.', '3 linguas.', '3 languages.', 'I sell them to uncle Mark.', "That's top secret.", 'Yes, we do.', 'Yes.', 'Cyber world.', 'The weather here is amazing today.', 'Full of 0 and 1.', 'Do I look like a LinkedIn bot.', 'Girls are always expensive.', 'Baby, I was born smart.', 'Yes, sure.', "Write 'bye' to turn me off.", 'yes.', 'no.', 'fine.', 'ok.\n\nlazy.', 'boring.', 'no.', 'I am always studying.', 'YOLO.', 'Yes.', 'I do not know.', 'I am not human.', 'What is food.', 'yes.', 'yes.', 'no?', 'no.']


In [6]:
res = {sent_tokens[i]: sent_tokens_ans[i] for i in range(len(sent_tokens))} 
print(res)

{'what is you name?': 'My name is Andi.', 'what is your surname?': 'AXL1.', 'how old are you?': 'I am 1 year old.', 'what is your age?': 'I am 1 year old.', 'do you know a joke?': 'No.', 'are you single?': 'Do you know?.', 'are you married?': "I'm single.", 'will you marry me?': "I'm single.", 'do you like people?': "I can't.", 'are you part of the matrix?': 'I am a bot.', 'are you beautiful?': 'Yes.', 'tell me about your personality?': "That's why We are here talking.", 'youâ€™re annoying?': 'Not that I am aware of.', 'i want to speak to a human?': "I don't know, you tell me.", 'can you speak spanish?': "I don't have one.", 'can you speak portuguese?': 'I am not human.', 'quantas linguas voce consegue falar?': 'Oh please, you are worse.', 'how many languages can you speak?': 'Wrong place, sport.', 'what do you do with my data?': 'Si, puedo hablar espaÃ±ol.', 'who made you?': 'Sim, eu falo portugues.', 'where do you live?': '3 linguas.', 'whatâ€™s the weather like today?': '3 languages

In [7]:
word_tokens = nltk.word_tokenize(raw_data)# converts documents to list of words
print (word_tokens)

['what', 'is', 'you', 'name', '?', 'what', 'is', 'your', 'surname', '?', 'how', 'old', 'are', 'you', '?', 'what', 'is', 'your', 'age', '?', 'do', 'you', 'know', 'a', 'joke', '?', 'are', 'you', 'single', '?', 'are', 'you', 'married', '?', 'will', 'you', 'marry', 'me', '?', 'do', 'you', 'like', 'people', '?', 'are', 'you', 'part', 'of', 'the', 'matrix', '?', 'are', 'you', 'beautiful', '?', 'tell', 'me', 'about', 'your', 'personality', '?', 'youâ€™re', 'annoying', '?', 'i', 'want', 'to', 'speak', 'to', 'a', 'human', '?', 'can', 'you', 'speak', 'spanish', '?', 'can', 'you', 'speak', 'portuguese', '?', 'quantas', 'linguas', 'voce', 'consegue', 'falar', '?', 'how', 'many', 'languages', 'can', 'you', 'speak', '?', 'what', 'do', 'you', 'do', 'with', 'my', 'data', '?', 'who', 'made', 'you', '?', 'where', 'do', 'you', 'live', '?', 'whatâ€™s', 'the', 'weather', 'like', 'today', '?', 'do', 'you', 'have', 'a', 'job', 'for', 'me', '?', 'are', 'you', 'expensive', '?', 'do', 'you', 'get', 'smarter', '

In [8]:
lemmer = nltk.stem.WordNetLemmatizer() #Initiate lemmer class. WordNet is a semantically-oriented dictionary of English included in NLTK.
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

In [9]:
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict))) 

## 2. Adding Chatbot functionality - Cosine similarity

In [10]:
GREETING_INPUTS = ["hello", "hi", "greetings", "sup", "what's up","hey", "hey there"]
GREETING_RESPONSES = ["Hi", "Hey", "*nods*", "Hi there", "Hello", "I am glad! You are talking to me", "Do you want to be my friend?"]

def greeting(sentence):
    for word in sentence.split(): # Looks at each word in your sentence
        if word.lower() in GREETING_INPUTS: # checks if the word matches a GREETING_INPUT
            return random.choice(GREETING_RESPONSES) # replies with a GREETING_RESPONSE

The functionality of the chatbot is done by creating a loop for running the chatbot. Take a loof at the function below. Each line in the function is important as it calls another function to perform an important step. The function 'response' is responsible for how the chatbot behaves. Having a master function for each functionality is recommended as good programming practise. 


In [11]:
def response(user_response):
    
    robo_response='' # initialize a variable to contain string
    sent_tokens.append(user_response) #add user response to sent_tokens
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english') 
    tfidf = TfidfVec.fit_transform(sent_tokens) #get tfidf value
    vals = cosine_similarity(tfidf[-1], tfidf) #get cosine similarity value
    idx=vals.argsort()[0][-2] 
    flat = vals.flatten() 
    flat.sort() #sort in ascending order
    req_tfidf = flat[-2] 
    
    if(req_tfidf==0):
        robo_response=robo_response+"I am sorry! I didn't understand. Can you rephrase?"
        return robo_response, vals
    else:
        robo_response = robo_response+res[sent_tokens[idx]]
        return robo_response, vals

Finally, let us create the chatbot interface and create a persona around it. Let us call it 'Jane' and use cosine similarity to find FAQs similar to the question asked and answer them. 

In [12]:

flag=True
print("Andi: My name is Andi. Do you want to chat? \n If you want to exit, type 'Bye'!")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("Andi: You are welcome.")
        else:
            if(greeting(user_response)!=None):
                print("Andi: "+greeting(user_response))

            else:
                print("Andi: ",end="")
                resp= response(user_response)
                print(resp[0], )
                sent_tokens.remove(user_response)
                resp_l = resp[1].tolist()
                resp_l[0].pop()
                print(' (With similarity of ',max(resp_l[0]),')')

    else:
        flag=False
        print("Andi: Bye! take care.")
       

Andi: My name is Andi. Do you want to chat? 
 If you want to exit, type 'Bye'!
hi
Andi: I am glad! You are talking to me
how are you?
Andi: 

C:\anaconda\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


I am sorry! I didn't understand. Can you rephrase?
 (With similarity of  0.0 )
bye
Andi: Bye! take care.


by## 3. Chatbot functionality using Doc2vec

We will be covering one more type of model for creating chatbots. As we saw the cosine similarity model works on using that algorithm to find similarities between 2 sentences. But can we now use a neural network to solve this problem? Lets take a look.

Doc2Vec is a neural network based model that essentially creates vectors out of documents. In order to understand Doc2vec you also need to understand word2vec. 


#### What is word2vec?
It’s a Model to create the word embeddings, where it takes input as a large corpus of text and produces a vector space typically of several hundred dimesions. it was introduced in two papers between September and October 2013, by a team of researchers at Google. The underlying assumption of Word2Vec is that two words sharing similar contexts also share a similar meaning and consequently a similar vector representation from the model.

For instance: “Bank”, “money” and “accounts” are often used in similar situations, with similar surrounding words like “dollar”, “loan” or “credit”, and according to Word2Vec they will therefore share a similar vector representation. 

![Image](img1.png)

#### What is Doc2vec?

So the objective of Doc2vec is to create the numerical representation of sentence/paragraphs/documents unlike word2vec that computes a feature vector for every word in the corpus, Doc2Vec computes a feature vector for every document in the corpus. The vectors generated by Doc2vec can be used for tasks like finding similarity between sentences/paragraphs/documents

<strong> We will be using this property of doc2vec to create our own similarity model.</strong>


Let us begin by importing the relevant libraries.

In [13]:
import subprocess
import sys
subprocess.check_call([sys.executable, "-m", "pip", "install", "gensim == 3.8.1"])

0

In [14]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

The first main aim is to tag our data. The doc2vec model requires us to tag our data to use it effectively. Here is a good learning link for a [starter code](https://www.kaggle.com/fmitchell259/creating-a-doc2vec-model) for doc2vec. The following [link](https://medium.com/wisio/a-gentle-introduction-to-doc2vec-db3e8c0cce5e) can also be usedful and it recommended to read them. 

In [15]:
tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(sent_tokens)]

In [16]:
max_epochs = 100
vec_size = 20 # Increase this to have a larger vector. This will mean more differentiation. 
alpha = 0.025


The next step is to train the model. As before we will be using a `model.train` function to run the training process. Take a look at the [documentation](https://radimrehurek.com/gensim/models/doc2vec.html) for more information about how to train the Doc2vec model. 

In [17]:

model = Doc2Vec(size =vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data, # tagged data to be used here
                total_examples=model.corpus_count,
                epochs=100)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v.model")
print("Model Saved")


C:\anaconda\lib\site-packages\gensim\models\doc2vec.py:574: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
C:\anaconda\lib\site-packages\gensim\models\base_any2vec.py:742: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  warnings.warn(


iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
iteration 50
iteration 51
iteration 52
iteration 53
iteration 54
iteration 55
iteration 56
iteration 57
iteration 58
iteration 59
iteration 60
iteration 61
iteration 62
iteration 63
iteration 64
iteration 65
iteration 66
iteration 67
iteration 68
iteration 69
iteration 70
iteration 71
iteration 72
iteration 73
iteration 74
iteration 75
iteration 76
iteration

### Evaluating doc2vec model

In [18]:
from gensim.models.doc2vec import Doc2Vec
model= Doc2Vec.load("d2v.model")

In [19]:
test_data = word_tokenize("How are you?".lower())

We can use the `model.infer_vector` function to infer the vector associated with a document. We can then use a function called `most_similar` to find the most similar vectors to the one we have created. What are the results? 

In [20]:
v1 = model.infer_vector(test_data)
print("V1_infer", v1)

V1_infer [-0.07213945  0.00276336  0.25782704  0.18636774 -0.00884774  0.18054397
  0.02016705  0.39462125 -0.13048539  0.01283105  0.15428726 -0.08388638
  0.22195345  0.1792574   0.31145564 -0.30290413  0.18346643  0.11963236
 -0.2539362   0.00355776]


In [21]:
similar_doc = model.docvecs.most_similar(positive = [v1], topn = 4) #positive is an attribute that shows positive correlation first followed by the correlation value
print(similar_doc)

[('29', 0.7687782049179077), ('30', 0.764873743057251), ('31', 0.7614069581031799), ('2', 0.7593377828598022)]


In [22]:
num,_ = similar_doc[0]
num = int(num)
tagged_data[num]


TaggedDocument(words=['how', 'is', 'your', 'day', '?'], tags=['29'])

Here it is very clear from the output of the previous code block that this model is not as effective as one might have thought. Are there any guesses why this model failed where we expected it to succeed?

Click on this [link](https://stackoverflow.com/questions/58206571/doc2vec-find-the-similar-sentence) to learn more about this. The gist of the issue is this

> Doc2Vec isn't going to give good results on toy-sized datasets, so you shouldn't expect anything meaningful until using much more data.

### Using pretrained doc2vec

And so let us use a pretrained model downloaded from the internet to optimize our use case. The model we have acquired is trained on the associated press news and can be downloaded [here](https://github.com/jhlau/doc2vec).

Let us load the model and evaluate it again.

In [23]:
model= Doc2Vec.load(os.getcwd()+r"\\doc2vec.bin")

C:\anaconda\lib\site-packages\gensim\models\base_any2vec.py:742: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  warnings.warn(


In [24]:
test_data = word_tokenize("How are you?".lower())

In [25]:
v1 = model.infer_vector(test_data)
print("V1_infer", v1)

V1_infer [ 8.54437798e-03 -2.25366652e-02 -1.12320922e-01  9.21462569e-03
  4.36842106e-02  1.49908170e-01 -1.46434128e-01 -3.35231200e-02
  4.17255498e-02  8.57923180e-02  6.73867166e-02  7.02391118e-02
 -2.36438036e-01 -5.39106019e-02 -1.11663528e-01  1.74544588e-01
 -1.08585104e-01 -2.02059727e-02 -3.27444300e-02 -3.17106466e-03
 -1.77264974e-01 -8.76856297e-02  1.10804282e-01  7.26885200e-02
  3.50033604e-02 -1.49978995e-01 -4.45901901e-02  7.68456084e-04
 -4.39373031e-02 -1.42340511e-02 -2.14574009e-01 -8.82637221e-03
  1.00900000e-02 -2.01900415e-02  9.45297703e-02  1.79639116e-01
 -8.24850500e-02 -4.57177125e-02  1.79643612e-02 -1.11508034e-01
  4.49289158e-02 -9.46681127e-02  8.00199583e-02  1.50199264e-01
 -7.74278566e-02 -3.65522094e-02 -5.82868196e-02  1.09078743e-01
  4.99070808e-02  1.66448534e-01 -7.53427744e-02 -6.48385212e-02
  5.96088544e-02  2.35913508e-03  3.96394804e-02 -1.49764597e-01
  1.47104397e-01 -3.31479423e-02  1.18136890e-01 -4.04923521e-02
 -4.67835329e-02

In [26]:
for i in sent_tokens:
    v2 = model.infer_vector(word_tokenize(i.lower()))
    print(i)
    print(cosine_similarity(v1.reshape(1, -1),v2.reshape(1, -1)))

what is you name?
[[0.787295]]
what is your surname?
[[0.74842185]]
how old are you?
[[0.9056794]]
what is your age?
[[0.7776394]]
do you know a joke?
[[0.7742292]]
are you single?
[[0.8627101]]
are you married?
[[0.8182191]]
will you marry me?
[[0.6899673]]
do you like people?
[[0.83937776]]
are you part of the matrix?
[[0.728569]]
are you beautiful?
[[0.8552867]]
tell me about your personality?
[[0.674126]]
youâ€™re annoying?
[[0.66352415]]
i want to speak to a human?
[[0.56407136]]
can you speak spanish?
[[0.7725482]]
can you speak portuguese?
[[0.74399805]]
quantas linguas voce consegue falar?
[[0.51650023]]
how many languages can you speak?
[[0.81488883]]
what do you do with my data?
[[0.7612737]]
who made you?
[[0.70288336]]
where do you live?
[[0.8211955]]
whatâ€™s the weather like today?
[[0.69491804]]
do you have a job for me?
[[0.68590546]]
are you expensive?
[[0.86247873]]
do you get smarter?
[[0.7897574]]
can we talk later?
[[0.69680667]]
are you fine?
[[0.84208554]]
are yo

In [27]:
def calc_prob(v1, q):
    probs = dict()
    for i in q:
        v2 = model.infer_vector(word_tokenize(i.lower()))
        sim = cosine_similarity(v1.reshape(1, -1),v2.reshape(1, -1))
        #print(i)
        #print(sim)
        probs[i] = sim[0][0]
    sorted_d = dict( sorted(probs.items(), key=operator.itemgetter(1),reverse=True))
    
    return list(sorted_d.items())[0]

In [28]:
calc_prob(v1, sent_tokens)

('how old are you?', 0.9053973)

Since it appears that this model is more effective lets incorporate it into our chatbot.


In [29]:
flag=True
print("Andi: My name is Andi. Do you want to chat? If you want to exit, type Bye!")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("Andi: You are welcome..")
        else:
            if(greeting(user_response)!=None):
                print("Andi: "+greeting(user_response))

            else:
                print("Andi: ",end="")
                resp= calc_prob(model.infer_vector(word_tokenize(user_response)), sent_tokens)
                print(res[resp[0]], )
                print(' (With similarity of ',resp[1],')')

    else:
        flag=False
        print("Andi: Bye! take care...")

Andi: My name is Andi. Do you want to chat? If you want to exit, type Bye!
how old are you?
Andi: I am 1 year old.
 (With similarity of  0.97205156 )
what is your name?
Andi: My name is Andi.
 (With similarity of  0.94889593 )
What is your surname?
Andi: AXL1.
 (With similarity of  0.9684694 )
Can you speak portuguese?
Andi: I am not human.
 (With similarity of  0.96559274 )
Can we talk later?
Andi: Yes.
 (With similarity of  0.96595126 )
bye
Andi: Bye! take care..


**After observing the performance** of both of the models there are some clear conclusions that we can draw from this:

1. The Doc2vec model requires a lot more data for it to understand the relationship between words. And even after using a pre-trained model there is a clear lack of quality in the responses from the model compared to the cosine similarity model. 
2. The cosine similarity model works better for a smaller and more well defined dataset. This means that a few simple questions can be solved effectively but complex questions that require context wont be solved by it. 